In [1]:
!pip install fhirpy

In [2]:
from fhirpy import SyncFHIRClient

In [3]:
client = SyncFHIRClient('https://server.fire.ly')


In [4]:

patient = client.resource(
    'Patient',
    name=[{'family': 'Sharma', 'given': ['Priya']}],
    gender='female',
    birthDate='1995-03-21',
    address=[{'line': ['123 Main St'], 'city': 'Mumbai', 'country': 'India'}],
    identifier=[{'system': 'http://hospital.smarthealth.org', 'value': 'PRIYA123'}],
    telecom=[{'system': 'phone', 'value': '+91-9876543210'}]
)
patient.save()
print(f"Patient created with ID: {patient['id']}")

Patient created with ID: d44d12a3-2205-4515-89e8-fa3094647a01


In [5]:
# Create a Condition resource for AML diagnosis without 'verificationStatus'
aml_condition = client.resource(
    'Condition',
    subject={'reference': f'Patient/{patient["id"]}'},
    code={'coding': [{'system': 'http://hl7.org/fhir/sid/icd-10', 'code': 'C92.0', 'display': 'Acute Myeloid Leukemia'}]},
    clinicalStatus={'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical', 'code': 'active'}]},
    onsetDateTime='2024-09-01'
)
aml_condition.save()
print(f"Condition created for AML with ID: {aml_condition['id']}")


Condition created for AML with ID: 3785ecb1-97aa-4b2c-98f9-2fee2ac53b68


In [6]:
# Step 4: Create an Observation resource for lab results
wbc_observation = client.resource(
    'Observation',
    status='final',
    category=[{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category', 'code': 'laboratory'}]}],
    code={'coding': [{'system': 'http://loinc.org', 'code': '6690-2', 'display': 'Leukocytes [#/volume] in Blood'}]},
    subject={'reference': f'Patient/{patient["id"]}'},
    valueQuantity={'value': 20000, 'unit': 'cells/uL', 'system': 'http://unitsofmeasure.org', 'code': '10*3/uL'},
    effectiveDateTime='2024-09-01'
)
wbc_observation.save()
print(f"WBC Observation created with ID: {wbc_observation['id']}")



WBC Observation created with ID: f6501b45-facd-4dd4-b12e-43cc623ca48a


In [7]:
# Step 5: Create a DiagnosticReport resource for the AML diagnostic tests
diagnostic_report = client.resource(
    'DiagnosticReport',
    status='final',
    code={'coding': [{'system': 'http://loinc.org', 'code': '34528-0', 'display': 'Bone marrow aspirate report'}]},
    subject={'reference': f'Patient/{patient["id"]}'},
    result=[{'reference': f'Observation/{wbc_observation["id"]}'}],
    conclusion="Acute Myeloid Leukemia (AML) confirmed",
    issued='2024-09-01T12:00:00Z'
)
diagnostic_report.save()
print(f"Diagnostic Report created with ID: {diagnostic_report['id']}")

Diagnostic Report created with ID: afa33e3e-7eb9-4125-969f-2aa6780e3712


In [8]:
# 5. Create a CarePlan for AML treatment (chemotherapy)
care_plan = client.resource(
    'CarePlan',
    status='active',
    intent='plan',
    description='AML treatment plan including chemotherapy',
    subject={'reference': f'Patient/{patient["id"]}'},
    period={'start': '2024-09-01', 'end': '2025-03-01'},
    activity=[{
        'detail': {
            'code': {'coding': [{'system': 'http://snomed.info/sct', 'code': '367336001', 'display': 'Chemotherapy regimen'}]},

            'status': 'not-started'
        }
    }]
)
care_plan.save()
print(f"CarePlan for chemotherapy created with ID: {care_plan['id']}")


CarePlan for chemotherapy created with ID: bf918ae3-6b75-44eb-902f-1237227cccb2


In [9]:
# 6. Create a CarePlan resource for chemotherapy treatment
care_plan = client.resource(
    'CarePlan',
    subject={'reference': f'Patient/{patient["id"]}'},
    status='active',
    intent='plan',
    title='Chemotherapy Plan for AML',
    description='Plan for induction chemotherapy as part of AML treatment',
    period={'start': '2024-09-01'},
    activity=[{
        'detail': {
            'code': {
                'coding': [{'system': 'http://snomed.info/sct', 'code': '367336001', 'display': 'Chemotherapy regimen'}]
            },
            'status': 'in-progress',
            'scheduledPeriod': {'start': '2024-09-01', 'end': '2024-10-01'}
        }
    }]
)
care_plan.save()
print(f"CarePlan for chemotherapy created with ID: {care_plan['id']}")


CarePlan for chemotherapy created with ID: 2d369f3d-b907-4be3-99e7-a0e7cec7efba


In [10]:
# 7. Create a CarePlan resource for chemotherapy treatment
care_plan = client.resource(
    'CarePlan',
    subject={'reference': f'Patient/{patient["id"]}'},
    status='active',
    intent='plan',
    title='Chemotherapy Plan for AML',
    description='Plan for induction chemotherapy as part of AML treatment',
    period={'start': '2024-09-01'},
    activity=[{
        'detail': {
            'code': {
                'coding': [
                    {'system': 'http://snomed.info/sct', 'code': '367336001', 'display': 'Chemotherapy regimen'}
                ]
            },
            'status': 'in-progress',
            'scheduledPeriod': {'start': '2024-09-01', 'end': '2024-10-01'}
        }
    }]
)

care_plan.save()
print(f"CarePlan for chemotherapy created with ID: {care_plan['id']}")


CarePlan for chemotherapy created with ID: 7f71a29a-b2cc-42e3-9d75-46af79021935


In [11]:
# Step 8: Retrieve and update Priya's AML condition to remission
condition = client.resources('Condition').search(_id=aml_condition['id']).first()

if condition:
    condition['clinicalStatus']['coding'][0]['code'] = 'remission'
    condition.save()
    print(f"Condition updated: {condition['clinicalStatus']['coding'][0]['code']}")
else:
    print("Condition not found")
observations = client.resources('Observation').search(subject=f'Patient/{patient["id"]}').fetch()

# Print each observation
for obs in observations:
    print(f"Observation ID: {obs['id']}, Value: {obs['valueQuantity']['value']} {obs['valueQuantity']['unit']}")



Condition updated: remission
Observation ID: f6501b45-facd-4dd4-b12e-43cc623ca48a, Value: 20000 cells/uL


# Downloading the json

In [12]:
import json
import os
from fhirpy import SyncFHIRClient
from google.colab import files

In [13]:
client = SyncFHIRClient('https://server.fire.ly')

In [14]:
patient_id = '0c3ec788-bac5-4ecf-9dd9-fd263f6ba8c0'
patient = client.resources('Patient').search(_id=patient_id).first()
if not patient:
    print(f"Patient with ID {patient_id} not found.")
else:
    print(f"Patient found: {patient['id']}")


Patient found: 0c3ec788-bac5-4ecf-9dd9-fd263f6ba8c0


In [15]:
save_directory = '/content'
os.makedirs(save_directory, exist_ok=True)

In [16]:

patient_id = patient['id']
patient = client.resources('Patient').search(_id=patient_id).first()
if not patient:
    print(f"Patient with ID {patient_id} not found.")
else:
    print(f"Patient found: {patient['id']}")
conditions = client.resources('Condition').search(subject=f'Patient/{patient_id}').fetch()
observations = client.resources('Observation').search(subject=f'Patient/{patient_id}').fetch()
diagnostic_reports = client.resources('DiagnosticReport').search(subject=f'Patient/{patient_id}').fetch()
care_plans = client.resources('CarePlan').search(subject=f'Patient/{patient_id}').fetch()

bundle = {
    "resourceType": "Bundle",
    "type": "collection",
    "entry": []
}
if patient:
    bundle["entry"].append({
        "fullUrl": f'Patient/{patient_id}',
        "resource": patient
    })
for condition in conditions:
    bundle["entry"].append({
        "fullUrl": f'Condition/{condition["id"]}',
        "resource": condition
    })
for observation in observations:
    bundle["entry"].append({
        "fullUrl": f'Observation/{observation["id"]}',
        "resource": observation
    })
for report in diagnostic_reports:
    bundle["entry"].append({
        "fullUrl": f'DiagnosticReport/{report["id"]}',
        "resource": report
    })
for care_plan in care_plans:
    bundle["entry"].append({
        "fullUrl": f'CarePlan/{care_plan["id"]}',
        "resource": care_plan
    })
import json
bundle_file_path = 'aml_patient_bundle.json'
with open(bundle_file_path, 'w') as bundle_file:
    json.dump(bundle, bundle_file, indent=4)
print(f"JSON bundle created and saved to {bundle_file_path}")


Patient found: 0c3ec788-bac5-4ecf-9dd9-fd263f6ba8c0
JSON bundle created and saved to aml_patient_bundle.json


In [17]:

patient_id = patient['id']
fetched_patient = client.resources('Patient').search(_id=patient_id).first()
if fetched_patient:
    print("Patient Details:")
    print(f"ID: {fetched_patient['id']}")
    print(f"Name: {', '.join([name['given'][0] + ' ' + name['family'] for name in fetched_patient['name']])}")
    print(f"Gender: {fetched_patient['gender']}")
    print(f"Birth Date: {fetched_patient['birthDate']}")
    print(f"Address: {', '.join([addr['line'][0] + ', ' + addr['city'] for addr in fetched_patient['address']])}")
    print(f"Contact Number: {', '.join([telecom['value'] for telecom in fetched_patient['telecom'] if telecom['system'] == 'phone'])}")
else:
    print("Patient not found on the server.")


Patient Details:
ID: 0c3ec788-bac5-4ecf-9dd9-fd263f6ba8c0
Name: Priya Sharma
Gender: female
Birth Date: 1995-03-21
Address: 123 Main St, Mumbai
Contact Number: +91-9876543210
